In [ ]:
# Long-Range Person Detection - Standard Training (YOLOv11)

This notebook runs the standard YOLOv11 training pipeline without SAHI.
It uses `pip` to install dependencies directly into the Colab environment.

In [ ]:
# 1. Clone the Repository
!git clone https://github.com/haroon0x/y11-rfdetr.git
%cd y11-rfdetr

In [ ]:
# 2. Install Dependencies
# In Colab, we use pip directly. We don't need 'uv' virtual environments here.
!pip install -r requirements.txt
!pip install ultralytics

In [ ]:
# 3. Verify Environment
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

In [ ]:
# 4. Run Training Pipeline
# Using optimized settings for Colab T4 GPU to avoid OOM
!python scripts/train_pipeline.py --epochs 50 --batch 8 --imgsz 960 --workers 2 --model yolo11m.pt

In [ ]:
# 5. Display Training Results (Graphs)
import os
from IPython.display import Image, display

# Find the latest training run directory
base_dir = "yolo11m_training_runs"
if os.path.exists(base_dir):
    steps = sorted(os.listdir(base_dir))
    if steps:
        latest_step = steps[-1]
        results_path = os.path.join(base_dir, latest_step, "results.png")
        confusion_matrix_path = os.path.join(base_dir, latest_step, "confusion_matrix.png")
        
        print(f"Displaying results for: {latest_step}")
        if os.path.exists(results_path):
            display(Image(filename=results_path))
        else:
            print("results.png not found.")
            
        if os.path.exists(confusion_matrix_path):
            display(Image(filename=confusion_matrix_path))
    else:
        print("No training steps found.")
else:
    print("Training output directory not found.")

In [ ]:
# 6. Run Benchmark
# Benchmarks the best model from the first step (VisDrone) on the validation set
!yolo benchmark model=yolo11m_training_runs/step1_visdrone/weights/best.pt data=VisDrone.yaml imgsz=640 half=True device=0

In [ ]:
# 7. Download Results
!zip -r runs_standard.zip yolo11m_training_runs/
from google.colab import files
files.download('runs_standard.zip')